In [ ]:
#import necessary libraries.
import numpy as np
import cv2

#initialize variable. 
load_from_disk = True
#if true then it loads the hsv value from the mamory.
if load_from_disk:
    hsv_value = np.load("hsv_value1.npy")
#initialize the web cam to capture the video.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
#initialize the kernel for morphological process.
kernel = np.ones((5,5),np.uint8)
#initialize the canvas over which we will draw.
canvas = None
#initialize x1, y1 points.
x1,y1 = 0,0
#threshold for noise.
noiseth = 800

while(1):
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    #initialize the canvas as a black image s the same size of the frame.
    if canvas is None:
        canvas = np.zeros_like(frame)
    #convert BGR TO HSV.
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    #assign the upper and lower values from the loaded hsv_value.
    if load_from_disk:
        lower_range = hsv_value[0]
        upper_range = hsv_value[1]
    #otherwise define the required values.
    else:
        lower_range = np.array([60, 100, 100])
        upper_range = np.array([179,255,255])
    #create mask for hsv image
    mask = cv2.inRange(hsv,lower_range,upper_range)
    #perform Morphological operation to get rid of noise.
    mask = cv2.erode(mask,kernel,iterations=1)
     
    mask =  cv2.dilate(mask,kernel,iterations=1)
    #find contours.
    contours,hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #make sure that the contour size is greater than thee noise.
    if contours and cv2.contourArea(max(contours,key = cv2.contourArea)) > noiseth:
        c = max(contours,key = cv2.contourArea)
        #boundingrect is used to get x2,y2,w,h values by bounding the area of intrest.
        x2,y2,w,h = cv2.boundingRect(c)
        #assign x2,y2 values to x1 and y1 values if there is no previous points are detected.
        #this is true when we writing for the first time or when the pen had disappeared from the view.
        if x1==0 and y1==0 :
            x1,y1=x2,y2
        else:
            #draw the line on the canvas by using the coordinates.
            canvas = cv2.line(canvas, (x1,y1),(x2,y2),[255,0,0],4)
        #after the line is drawn the new position becomes the previous position.
        x1,y1=x2,y2
    else:
        #if no contours were detected then initialize them as zero.
        x1,y1=0,0
    # merge canvas and frame.        
    frame =cv2.add(frame,canvas)
    #optionally stack boyh frames and show it.
    stacked = np.hstack((canvas,frame))
    cv2.imshow("virtual pen",cv2.resize(stacked,None,fx=0.5,fy=0.5))
    k=cv2.waitKey(1)
    if k == 27:
        break
    #if C is pressed , clear the canvas.
    if k == ord('c'):
        canvas=None
        
cap.release()
cv2.destroyAllWindows()
            
    
        
        
        
        
        
        
        
        
        
        
        
        
        